In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

In [3]:
response = llm.invoke("Hello! Can you tell me a short joke in indian context about startup founders?")
print(response.content)

Here's one:

Why did the Indian startup founder bring a ladder to the meeting?

Because he wanted to take his valuation to the next level, and also to reach the "unicorn" status, but his investor told him "beta, pehle seed funding toh paas karo!" (first, at least pass the seed funding!)


### Search node

In [4]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)
web_search_results = web_search.invoke("who is the mayor of NYC?")

print(web_search_results["results"][0]["content"])

Jump to content ## Contents * 1 History * View history * View history * Download QR code * Download as PDF The mayor's office is located in New York City Hall; it has jurisdiction over all five boroughs "Borough (New York City)") of New York City: Manhattan, Brooklyn, the Bronx, Staten Island and Queens. According to current law, the mayor is limited to two consecutive four-year terms in office but may run again after a four-year break. The current mayor is Eric Adams, who was elected on November 2, 2021, and took office shortly after midnight on January 1, 2022. ## History Its last political leader was an African American man named J. (**Discuss**) *(June 2025)* | * Robert W. * Staten Island Historical Society


In [5]:
from langgraph.prebuilt import create_react_agent

groq_llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
)

# 2. Create BetterMeals research agent using Groq model
research_agent = create_react_agent(
    model=groq_llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)


In [6]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [7]:
for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "who is the mayor of NYC?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  tavily_search (fc_bac52efd-80a0-4d68-8e3e-51379ca35039)
 Call ID: fc_bac52efd-80a0-4d68-8e3e-51379ca35039
  Args:
    query: current mayor of New York City 2025
    search_depth: advanced


Update from node tools:


================================= Tool Message =================================
Name: tavily_search

{"query": "current mayor of New York City 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.nytimes.com/interactive/polls/nyc-mayoral-election-polls-2025.html", "title": "New York City Mayoral Election 2025: Latest Polls", "content": "With all the current candidates in the running, Zohran Mamdani holds nearly a two-to-one lead, 46 percent to 24 percent, over Andrew Cuomo, according to the New York Times/Siena University poll of the race. But if Cuomo got a one-on-one race against Mamdan

In [8]:
from typing import TypedDict, Optional

class State(TypedDict):
    raw_message: str
    query_type: Optional[str]
    response: Optional[str]


In [12]:
def handle_meal_doubt(state: dict):
    """Answer nutrition/meal-related user doubts."""
    return {"response": f"[MealDoubtAgent] Answering: {state['raw_message']}"}

def handle_user_update(state: dict):
    """Update user preferences or nutrition plan."""
    return {"response": f"[UserUpdateAgent] Updating plan with: {state['raw_message']}"}

def handle_cook_update(state: dict):
    """Handle cook updates like missing ingredients."""
    return {"response": f"[CookUpdateAgent] Cook update received: {state['raw_message']}"}

def handle_grocery_order(state: dict):
    """Trigger or update grocery orders."""
    return {"response": f"[GroceryOrderAgent] Ordering groceries: {state['raw_message']}"}


In [13]:
supervisor_agent = create_react_agent(
    model=groq_llm,
    tools=[
        handle_meal_doubt,
        handle_user_update,
        handle_cook_update,
        handle_grocery_order,
    ],
    prompt=(
        "You are the Supervisor Agent for BetterMeals.\n"
        "- Route meal/nutrition doubts to MealDoubtAgent.\n"
        "- Route user profile or preference changes to UserUpdateAgent.\n"
        "- Route cook updates (missing ingredients) to CookUpdateAgent.\n"
        "- Route grocery ordering requests to GroceryOrderAgent.\n"
        "Always assign tasks to the correct agent. Do not solve them yourself."
    ),
    name="supervisor",
)

In [15]:
for chunk in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": "order brocoli for dinner?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  handle_grocery_order (fc_2cbf950c-36b7-4d94-ad8c-1575cf59b17f)
 Call ID: fc_2cbf950c-36b7-4d94-ad8c-1575cf59b17f
  Args:
    state: {}


Update from node tools:


================================= Tool Message =================================
Name: handle_grocery_order

Error: KeyError('raw_message')
 Please fix your mistakes.


Update from node agent:


================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  handle_grocery_order (fc_67901d26-fe06-4fdd-a7f1-a1466b125f0b)
 Call ID: fc_67901d26-fe06-4fdd-a7f1-a1466b125f0b
  Args:
    state: {'items': ['broccoli']}


Update from node tools:


================================= Tool Message =================================
Name: handle_grocery_order

Error: KeyError('raw_message')
 Please fix your mistakes.


Update from node agent:


=============